# CHAPTER 09. 추천 시스템

## 05. 콘텐츠 기반 필터링 실습 - TMDB 5000 영화 데이터 세트

### 장르 속성을 이용한 영화 콘텐츠 기반 필터링
" 사용자가 특정 영화를 감상하고 그 영화를 좋아했다면 그 영화와 비슷한 특성/속성, 구성 요소 등을 가진 다른 영화를 추천하는 것이다. 이렇게 영화(또는 상품/서비스) 간의 유사성을 판단하는 기준이 영화를 구성하는 다양한 콘텐츠(장르, 감동, 배우, 평점 등)를 기반으로 하는 방식이 바로 콘텐츠 기반 필터링이다.
- 장르 칼럼 값의 유사도를 비교한 뒤 그중 높은 평점을 가지는 영화를 추천하는 방식이다.

### 데이터 로딩 및 가공

In [12]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

movies = pd.read_csv('/Users/wizdom/Desktop/data_analysis/파이썬 머신러닝 완벽가이드/실습 데이터/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporatio...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United ...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800


4803개의 레코드와 20개의 피처로 구성. 영화 제목, 예산, 장르, 등 영화에 대한 다양한 메타 정보를 가짐

In [13]:
# 콘텐츠기반 필터링 추천 분석에 사용할 주요 칼럼만 추출해 dataframe 만들기

movies_df = movies[['id', 'title', 'genres', 'vote_average', 'vote_count',
                    'popularity', 'keywords', 'overview']]

In [14]:
# 리스트 내부에 여러개의 딕셔너리가 있는 형태의 문자열인 두 칼럼 확인

pd.set_option('max_colwidth', 100)
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


genres 칼럼은 여러 개의 개별 장르 데이터를 가지고 있고, 개별 장르의 명칭은 딕셔너리의 키인 'name'으로 추출할 수 있다. keywords 칼럼 역시 마찬가지 구조를 가지고 있다.

In [15]:
# 두 칼럼의 문자열 분해하여 개별 장르를 리스트 객체로 추출

from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

이제 두 칼럼은 문자열이 아닌 실제 리스트 내부에 여러 장르 딕셔너리로 구성된 객체를 가진다.
- **ast 모듈의 literal_eval()** : 문자열이 의미하는 list[dict1, dict2]객체로 만든다.

In [16]:
# 장르명만 리스트 객체로 추출

movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


### 장르 콘텐츠 유사도 측정
1. 문자열로 변환된 genres칼럼을 Count 기반으로 피처 벡터화 변환
2. genres 문자열을 피처 벡터화 행렬로 변환한 데이터 세트를 코사인 유사도를 통해 비교. 이를 위해 데이터 세트의 레코드별로 타 레코드 장르에서 코사인 유사도 값을 가지는 객체 생성
3. 장르 유사도가 높은 영화 중에 평점이 높은 순으로 영화 추천

#### 1) genres 칼럼을 문자열로 변환한 뒤, CountVectorizer를 이용해 피처 벡터 행렬로 만들기

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
print(movies_df['genres_literal'])

0       Action Adventure Fantasy Science Fiction
1                       Adventure Fantasy Action
2                         Action Adventure Crime
3                    Action Crime Drama Thriller
4               Action Adventure Science Fiction
                          ...                   
4798                       Action Crime Thriller
4799                              Comedy Romance
4800               Comedy Drama Romance TV Movie
4801                                            
4802                                 Documentary
Name: genres_literal, Length: 4803, dtype: object


In [18]:
count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


#### 2) 코사인 유사도 계산
- **cosine_similarity()** : 기준 행과 비교 행의 코사인 유사도를 행렬 형태로 반환

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:1])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]]


genre_sim 객체는 movies_df의 genre_literal 칼럼을 피처 벡터화한 행렬 데이터의 행별 유사도 정보를 가지고 있으며, 결국은 Movies_df DataFrame의 행별 장르 유사도 값을 가지고 있다.

#### 3) 장르 유사도가 높은 순으로 레코드 추출
genre_sim 객체의 기준 행별로 비교 대상이 되는 행의 유사도 값이 높은 순으로 정렬된 행렬의 위치 인덱스 값 추출

In [20]:
# 유사도가 높은 순으로 정리된 genre_sim 객체의 비교 행 위치 인덱스 값 얻기
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
# 0번 레코드의 비교 행 위치 인덱스 값 샘플로 추출
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


0번 레코드의 경우, 자신인 0번 레코드를 제외하면 3494번 레코드가 가장 유사도가 높고 그다음 813레코드이며 가장 유사도가 낮은 레코드는 2401번 레코드라는 뜻이다.

### 장르 콘텐츠 필터링을 이용한 영화 추천

In [21]:
# 장르 유사도에 따라 영화를 추천하는 함수 생성

def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    # 인자로 입력된 movies_df DataFrame에서 'title' 칼럼이 입력된 title_name 값인 DataFrame 추출
    title_movie = df[df['title'] == title_name]
    
    # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index 출력. top_n index는 2차원 데이터임
    # dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [24]:
# '대부'와 장르별로 유사한 영화 10개 추천
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


'대부 2편'이 가장 먼저 추천됐다. 'Goodfellas'도 대부와 비슷한 유형으로 추천되었다. 하지만 'Light Sleeper', 'Mi America', 'Kids' 등 낯선 영화도 많다. 대부를 좋아하는 고객에게 섣불리 추천하기에는 이해하기 어려운 영화이다. 개선이 필요해 보인다.

#### 좀 더 많은 후보군을 선정한 뒤에 영화의 평점에 따라 필터링해서 최종 추천하는 방식으로 변경

In [25]:
# vote_average는 1명, 2명의 소수의 관객이 특정 영화에 매우 높은 평점을 부여해 왜곡된 데이터를 가지고 있다. 이를 확인
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


평가 횟수가 매우 작은 영화들이 있다. 이와 같은 왜곡된 평점 데이터를 회피할 수 있도록 평가 횟수를 반영할 수 있는 새로운 평가 방식이 필요하다.

IMDB에서는 평가 횟수에 대한 가중치가 부여된 평점 방식을 사용한다.
#### 가중 평점(Weighted Rating) = (v/(v+m)) * R + (m/(v+m)) * C
- 변수의 의미
    - **v** : 개별 영화에 평점을 투표한 횟수
    - **m** : 평점을 부여하기 위한 최소 투표 횟수(투표 횟수에 따른 가중치를 직접 조절하는 역할)
        - m값을 높이면 평점 투표 횟수가 많은 영화에 더 많은 가중 평점을 부여한다.
    - **R** : 개별 영화에 대한 평균 평점
    - **C** : 전체 영화에 대한 평균 평점

In [26]:
# C값 구하기
C = movies_df['vote_average'].mean()
# m값 구하기 
m = movies_df['vote_count'].quantile(0.6)
print('C:', round(C, 3), 'm:', round(m, 3))

C: 6.092 m: 370.2


In [29]:
# 기존 평점을 새로운 가중 평점으로 변경하는 함수 생성
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( ( v/(v+m)) * R) + ((m/(m+v)) * C)

# 새로운 평점 정보 칼럼
movies_df['weighted_vote'] = movies.apply(weighted_vote_average, axis=1)

In [30]:
# wieghted_vote 평점이 높은 순으로 상위 10개의 영화 추출
movies_df[['title', 'vote_average', 'weighted_vote', 'vote_count']].sort_values('weighted_vote',
                                                                               ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [34]:
# 새롭게 정의된 평점 기준에 따라 영화 추천

def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    # top_n의 2배에 해당하는 장르 유사성이 높은 인덱스 추출
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
    # 기준 영화 인덱스는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote가 높은 순으로 top_n만큼 추출
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427


이전에 추천된 영화보다 훨씬 나은 영화가 추천됐다.